In [5]:
import httpx
import asyncio

client =  httpx.AsyncClient()
async def fetch(url, headers, semaphore, client, timeout, max_retries, start_retry_delay):
    async with semaphore:
        async with client:
            for attempt in range(max_retries):
                try:
                    response = client.get(url, timeout=timeout, headers= headers)
                    response.raise_for_status()
                    return response  # Successful request, exit the loop
                except httpx.HTTPStatusError as e:
                    headers = response.headers
                    retry_after = headers.get('Retry-After')
                    if retry_after != None:
                        print(retry_after)
                        asyncio.sleep(retry_after.astype(int))
                        continue
                    print(f"Error response {e.response.status_code}.")
                except httpx.TimeoutException as e:
                    print(f"Timeout reached: {e}")
                    if attempt < max_retries:
                        print(f"Retrying in {attempt*start_retry_delay} seconds...")
                        asyncio.sleep(attempt*start_retry_delay)
                    else:
                        print("Max retries reached. Exiting.")
                        return 0
                except httpx.RequestError as e:
                    print(f"An error occurred: {e}.")



asyncio.gather()

<_GatheringFuture pending>

bla
bla
bla
bla


In [ ]:
import requests
import numpy as np
import json
import pandas as pd
import time
import yahoo_fin.stock_info as si
from datetime import datetime
from fredapi import Fred

fred = Fred(api_key='0c34c4dd2fd6943f6549f1c990a8a0f0') 


def getcik(ticker):
    #Convert the ticker into the proper cik
    for key,value in cikdata.items():
        if value["ticker"] == ticker:
            cik = value["cik_str"]
            break
    return str(cik).zfill(10)

#Headers for EDGAR call
headers = {
    "User-Agent":"ficakc@seznam.cz",
    "Accept-Encoding":"gzip, deflate",
}

TIMEOUT = 8
RETRIES = 2
START_RETRY_DELAY = 0.4
# cik_url =  "https://www.sec.gov/files/company_tickers.json"
# cikdata = requests.get(cik_url, headers=headers).json()

with open(r"C:\Programming\Python\Finance\EDGAR\cik.json","r") as file:
    cikdata = json.load(file)
    file.close()
with open(r"C:\Programming\Python\Finance\EDGAR\apple.json","r") as file:
    Apple = json.load(file)
    file.close()



def companyfacts(ticker:str, client, semaphore):
    #Get all the financial data for a ticker
    cik = getcik(ticker)
    data_url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"
    facts = fetch(data_url, headers, semaphore, client, TIMEOUT,RETRIES,START_RETRY_DELAY)
    return facts

def endtodatetime(dataframe):
    dataframe.loc[:,"end"] = pd.datetime(dataframe["end"])
    return dataframe

def fred_info(ids:list, start, end):
    #start and end are datatime objects
    start = start.strftime('%Y-%m-%d')
    end = end.strftime('%Y-%m-%d')
    frame = pd.DataFrame()
    for id in ids:
        series = fred.get_series(id,observation_start=start, observation_end=end)
        frame[id] = series
    frame = frame.reset_index()
    frame["index"] = frame["index"].astype(str)
    return frame.ffill().bfill()

class Stock:
    def __init__(self, ticker:str, measures:list):
        self.ticker = ticker.upper()
        self.cik = getcik(self.ticker)
        
    async def async_init(self,client,semaphore):
        data = await companyfacts(self.ticker, client, semaphore)
        #If the response wasn't recieved, skips the rest of the code 
        if type(data) != int:
            self.data = data.json()
        else:
            return
        share_amount = datetime.strptime(self.data["facts"]["dei"]["EntityCommonStockSharesOutstanding"]["units"]["shares"][0]["end"], r"%Y-%m-%d") if "EntityCommonStockSharesOutstanding" in self.data["facts"]["dei"] else datetime.strptime(self.data["facts"]["dei"]["EntityPublicFloat"]["units"]["shares"][0]["end"], r"%Y-%m-%d")
        self.start_year = max([share_amount] + [datetime.strptime(self.data["facts"]["us-gaap"][measure]["units"]["USD"][0]["end"], r"%Y-%m-%d") if measure in self.data["facts"]["us-gaap"] else datetime.strptime('1920-01-01', r"%Y-%m-%d") for measure in measures])
        self.end_year = datetime.now().date()
        #Get the price and set the self.price
        self.fullprice = si.get_data(self.ticker,self.start_year, self.end_year).reset_index()
        Price = self.fullprice[[self.fullprice.columns[0],"close", "adjclose"]].copy()
        Price["end"] = Price["index"].astype(str)
        Price.drop(columns=["index"],inplace=True)
        date_range = pd.date_range(start=self.start_year, end=self.end_year).astype(str)
        self.date_range = pd.DataFrame(date_range, columns=['end'])
        Price = pd.merge(self.date_range, Price, on = ["end"],how="left" )
        self.price = Price.bfill()

    def fact(self,measure,simple=True):
        #Propagate the 0 
        if self.data == 0:
            return 0
        try:
            point_list = self.data["facts"]["us-gaap"][measure]["units"]["USD"]
            frame = pd.DataFrame(point_list)
            frame = frame.drop_duplicates(subset='end', keep='last')
            frame[measure] = frame["val"]
            if simple:
                frame = frame[["end", measure]]
            frame = pd.merge(self.date_range,frame,on="end",how="left")
            return frame.ffill()
        except KeyError:
            print(f"Measure {measure} not available for company.")
    def shares(self,simple=True):
        #Propagate the 0 
        if self.data == 0:
            return 0
        try:
            if simple:
                share_count = pd.DataFrame(self.data["facts"]["dei"]["EntityCommonStockSharesOutstanding"]["units"]["shares"])[["end","val"]]
        except KeyError:
            if simple:
                share_count = pd.DataFrame(self.data["facts"]["dei"]["EntityPublicFloat"]["units"]["shares"])[["end","val"]]
        share_count["shares"] = share_count["val"]
        share_count.drop(columns=["val"], inplace = True)
        share_count = share_count.drop_duplicates(subset="end", keep="last")
        share_count = pd.merge(self.date_range, price, on=["end"], how="left")
        return share_count.ffill()
    
async def async_task(cik, client, semaphore, measures, indicators):
    print(f"Currently pinging {cik}")
    stock = Stock(cik, measures)
    stock.async_init(stock,client,semaphore)
    try:
        #Price and shares oustanding 
        shares = stock.shares().copy()
        stock_price = stock.price.copy()
        df = pd.merge(shares, stock_price, left_on=["end"], right_on=["end"], how = "left")
        frames_list = [stock.fact(measure) for measure in measures]
        for frame in frames_list:
            df = pd.merge(df,frame, on=["end"], how="left")
    #HANDE RETURNING NONES IN THE GATHER///////////////////////////
    except AttributeError:
        return
    #Economic indicators 
    indicator_frame = fred_info(indicators, stock.start_year, stock.end_year)
    df = pd.merge(df, indicator_frame, left_on =["end"], right_on=["index"], how="left")
    return (cik,df)


In [4]:
#write out measures based on importance in descending order
measures = ["Assets", "Liabilities", "AssetsCurrent", "LiabilitiesCurrent"]
#write out the indicators 
indicators = ["TB3MS", "DCOILWTICO"]
#Get the first n companies sorted by market cap 
companies_num = 3
comp = 0
#dict of all the companies' dataframes
frame_dict = {}
sem = asyncio.semaphore(9)
#GATHER THE FIRST companies_num companies ciks and pass them to the gather with the tasks
cik_list = []
for company, values in cikdata.items():
    if comp<companies_num:
        cik_list.append(values["cik_str"])
        comp+=1
    else:
        break
company_frames_tuples = asyncio.gather(*[async_task(cik) for cik in cik_list])
company_frames_dict = {}
for cik, df in company_frames_tuples:
    company_frames_dict[cik] = df

    
    

Currently pinging AAPL


Request failed: HTTPSConnectionPool(host='data.sec.gov', port=443): Read timed out. (read timeout=10)
Max retries reached. Exiting.
Currently pinging MSFT
             end        shares
0     2010-06-30           NaN
1     2010-07-01           NaN
2     2010-07-02           NaN
3     2010-07-03           NaN
4     2010-07-04           NaN
...          ...           ...
4942  2024-01-10  7.432262e+09
4943  2024-01-11  7.432262e+09
4944  2024-01-12  7.432262e+09
4945  2024-01-13  7.432262e+09
4946  2024-01-14  7.432262e+09

[4947 rows x 2 columns]
Currently pinging GOOGL
Request failed: HTTPSConnectionPool(host='data.sec.gov', port=443): Read timed out. (read timeout=10)
Max retries reached. Exiting.
Currently pinging AMZN
Request failed: HTTPSConnectionPool(host='data.sec.gov', port=443): Read timed out. (read timeout=10)
Max retries reached. Exiting.
Currently pinging NVDA
Currently pinging META


KeyError: 'shares'

In [ ]:
# Apple.data["facts"]["dei"]["EntityCommonStockSharesOutstanding"]["units"]["shares"]
frame  = fred_info(["TB3MS", "DCOILWTICO"], '2015-02-24', '2017-02-24')
frame.head(40)
# print(frame)

frame["index"] = frame["index"].astype(str)
with open("C:\Programming\Python\Finance\EDGAR\FRED.json", "w") as file:
    json.dump(frame.to_dict(orient="records"), file, indent=1)
    file.close()


In [ ]:
MSFT = companyfacts('MSFT').json()
print(MSFT["facts"]["dei"]["EntityCommonStockSharesOutstanding"]["units"]["shares"][0]["end"])


In [ ]:
measures = ["Assets", "Liabilities", "AssetsCurrent", "LiabilitiesCurrent"]
Apple = Stock("aapl", measures)
stock = Apple
shares = stock.shares()
stock_num = stock.price
if isinstance(shares, int) or isinstance(stock_num, int): 
    pass
    # break
df = pd.merge(shares.copy(), stock_num.copy(), on=["end"], how = "left")
frames_list = [stock.fact(measure) for measure in measures]
for frame in frames_list:
    df = pd.merge(df,frame, on=["end"], how="left")
df.head(16)

Path to info: Stock["facts"]["us-gaap"][some_measure]["units"]["USD"]

Path to meta info: Stock["facts"]["dei"]

Three things:
    First:
        Graph and correlate measures and ratios
            Currently forward filling fundamentals data, could be fucked in the future
            Include the rate of change of the variables and compare those 
    Second:
        Make a test to validate trading strategies
            Add a function that picks eligible stocks at some timeframe and test their performance xdz
    Third:
        Train a model to predict the long term price
            forward filling fundamentals can be really fucked 
            Consider using averages or linear change from previous value to next for training 
            Include all the past fundamentals for each datapoint - this could figure out how people value growth.

Data to add for regression and AI:
    oil price
    interest rates 
        FRED HAS SO MUCH DATA HOLY SHIT 
        pick yer fucking poison mate
    

Measures:
    Price/BookValuepershare = MarketCap/BookValue = EntityCommonStockSharesOutstanding*SPrice(Assets-Liabilities)
    Price/Earnings - all the possible earnings metrics
    Current ratio = AssetsCurrent/LiabilitiesCurrent
    ///////////////////////////////////////////////////////////////////////////////////////////////////////////
    Revenue
    Operating Income 
    Free cash flow 
    EBITDA 
    Dividends - "PaymentsOfDividendsCommonStock"

How does market cap influence these correlations?

Where do you find price data?
    Yahoo-fin.


Methods to figure out relationships:
    Kendall's Tau
    Spearman's Rank Correlation

    cross variable correlation loop through all variables 
    Polynomial regression
    LSTM


Todo:
    Investigate why not forward filling
    Fix Asynchronousness
        Implement asynchrnousness for FRED and Yahoofin.
    Implement ConnectionError safeguard

*Get all the measure names*

In [ ]:
# with open("C:\Programming\Python\Finance\EDGAR\stock.json", "w") as file:
#     json.dump(df.to_dict(orient='records'), file, indent=1)
#     file.close()

with open("C:\Programming\Python\Finance\EDGAR\shares.json", "w") as file:
    json.dump(stock.shares().copy().to_dict(orient='records'), file, indent=1)
    file.close()

with open(r"C:\Programming\Python\Finance\EDGAR\assets.json", "w") as file:
    json.dump(Apple["facts"]["us-gaap"]["Assets"]["units"]["USD"], file, indent=1)
    file.close()


    

In [ ]:
measures = {}
for key,value in Apple["facts"]["us-gaap"].items() :
    measures[key.ljust(100)] = value["label"]

measures["METADATA".ljust(200,"/")] = ""

for key,value in Apple["facts"]["dei"].items():
    measures[key.ljust(100)] = value["label"]


with open(r"C:\Programming\Python\Finance\EDGAR\measures.json","w") as file:
    json.dump(measures, file, indent=1)


#create price reference list:
with open("C:\Programming\Python\Finance\EDGAR\price.json", "w") as file:
    json.dump(Apple.price.to_dict(orient='records'), file, indent=1)
    file.close()


In [ ]:
for key in Apple["facts"]["us-gaap"]["AssetsCurrent"]["units"]["USD"] :
    try:
        del key["frame"]
        print(key)
    except KeyError:
        print(key)



In [ ]:
# print(Apple["facts"]["us-gaap"]["Assets"]["units"]["USD"][0]["end"])
